In [1]:
import tensorflow as tf
import PIL
import os
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np


### IMAGE PREPARATION

In [2]:
IMAGE_SIZE = 28
if 'data' not in os.listdir('./'):
    os.mkdir('./data')
for i in range(1, 9):
    if str(i) not in os.listdir('./data'):
        os.mkdir('./data/'+str(i)+'/')
    for j in range(1, 21):
        
        file_name = 'handwritten_data/{directory}/{file}.jpg'.format(directory=i, file=j)
        file_save = 'data/{directory}/{file}.jpg'.format(directory=i, file=j)
        image = Image.open(file_name)
        
        new_image = np.array(image.resize((IMAGE_SIZE, IMAGE_SIZE), PIL.Image.ANTIALIAS))
        plt.imsave(file_name, new_image)
        
        im = Image.fromarray(plt.imread(file_name)[:, :, :3])
        im.save(file_save)

['.ipynb_checkpoints', 'data_augmentation.ipynb', 'handwritten_data', 'model', 'test_loaded_model.ipynb', 'training.ipynb']


### AUGMENTATION

#### Rotating

In [3]:
scaling_degrees = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

for i in range(1, 9):
    for j in range(1, 21):
        for degree in scaling_degrees:
            file_name = 'handwritten_data/{directory}/{file}.jpg'.format(directory=i, file=j)
            tmp_image = Image.open(file_name)
            tmp_image = tmp_image.rotate(degree, fillcolor='white')
            tmp_image.save('data/{directory}/{file}_rotated_{degree}.jpg'.format(directory=i, file=j, degree=degree))

In [4]:
scaling_degrees = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

for i in range(1, 9):
    for j in range(1, 21):
        for degree in scaling_degrees:
            file_name = 'handwritten_data/{directory}/{file}.jpg'.format(directory=i, file=j)
            tmp_image = Image.open(file_name)
            tmp_image = tmp_image.rotate(360-degree, fillcolor='white')
            tmp_image.save('data/{directory}/{file}_rotated_{degree}.jpg'.format(directory=i, file=j, degree=360-degree))

### Cropping

In [5]:
IMAGE_SIZE = 28

def crop_image(X_img, crop):
    tf.reset_default_graph()
    
    original_size = [IMAGE_SIZE, IMAGE_SIZE, 3]
    X = tf.placeholder(dtype = tf.float32, shape = original_size)

    crop_size = [crop, crop, 3]
    
    seed = np.random.randint(7)
    X_crop = tf.random_crop(X, size = crop_size, seed = seed)
    output = tf.image.resize_images(X_crop, size =(IMAGE_SIZE, IMAGE_SIZE))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        cropped = sess.run(output, feed_dict = {X: X_img})
        cropped = tf.cast(cropped, np.uint8).eval()
    return cropped

In [6]:
cropped_size = [23, 22, 21, 20]

for i in range(1, 9):
    for j in range(1, 21):
        for crop in cropped_size:
            file_name = 'handwritten_data/{directory}/{file}.jpg'.format(directory=i, file=j)
            tmp_image = mpimg.imread(file_name)
            cropped_image = crop_image(tmp_image, crop)
            mpimg.imsave('data/{directory}/{file}_cropped_{crop}.jpg'.format(directory=i, file=j, crop=crop), cropped_image)

### Translation

In [7]:
vertical_translation = [1, 2, 3, 4, -1, -2, -3, -4]

for i in range(1, 9):
    for j in range(1, 21):
        for pad in vertical_translation:
            file_name = 'handwritten_data/{directory}/{file}.jpg'.format(directory=i, file=j)
            tmp_image = Image.open(file_name)
            translated_image = tmp_image.transform(tmp_image.size, Image.AFFINE, (1, 0, 0, 0, 1, pad), fillcolor='white')
            mpimg.imsave('data/{directory}/{file}_translated_{pad}.jpg'.format(directory=i, file=j, pad=pad), 
                         np.array(translated_image))